In [176]:
import pandas as pd
import os
# os.chdir("../")
data_ke_3=pd.read_csv("../OTC/ingre_cate_len3.csv",index_col=0,encoding="shift-jis")
data_ke_2=pd.read_csv("../OTC/ingre_cate_len2.csv",index_col=0,encoding="shift-jis")
data_yaku=pd.read_csv("../yakuji/monthly/prod_joined_monthly.csv",encoding="shift-jis",index_col=0)
data_ship=pd.read_csv("../yakuji/monthly/ship_joined_monthly.csv",encoding="shift-jis",index_col=0)

In [177]:
data_yaku.columns

Index(['year', 'month', 'id', 'efficacy', 'rx_agg', 'rx_pro_agg', 'rx_pro_dom',
       'otc_agg', 'otc_pro_agg', 'otc_pro_dom', 'self_agg', 'self_pro_agg',
       'self_pro_dom'],
      dtype='object')

In [178]:
data_yaku.replace(' ',"0",inplace=True)
data_yaku["id"]=data_yaku["id"].astype(int) 
data_yaku['month'] = data_yaku['month'].astype(int)
data_yaku['year'] = data_yaku['year'].astype(int)
# same for ship
data_ship.replace(' ',"0",inplace=True)
data_ship["id"]=data_ship["id"].astype(int)
data_ship['month'] = data_ship['month'].astype(int)
data_ship['year'] = data_ship['year'].astype(int)
data_yaku['year_month'] = data_yaku.apply(lambda row: f"{row['year']}-{row['month']:02d}", axis=1)
data_ship['year_month'] = data_ship.apply(lambda row: f"{row['year']}-{row['month']:02d}", axis=1)

In [179]:
data_yaku.columns, data_ship.columns

(Index(['year', 'month', 'id', 'efficacy', 'rx_agg', 'rx_pro_agg', 'rx_pro_dom',
        'otc_agg', 'otc_pro_agg', 'otc_pro_dom', 'self_agg', 'self_pro_agg',
        'self_pro_dom', 'year_month'],
       dtype='object'),
 Index(['year', 'month', 'id', 'efficacy', 'rx_dom', 'otc_dom', 'self_dom',
        'stock_rx', 'stock_otc', 'stock_self', 'year_month'],
       dtype='object'))

In [180]:
len(set(data_yaku["year_month"])),len(set(data_ship["year_month"])),len(set(data_ship["id"])),len(set(data_yaku["id"]))

(172, 172, 209, 209)

In [181]:
data_yaku.shape,data_ship.shape

((34486, 14), (34687, 11))

In [182]:
data_ship[(data_ship["id"]==611) & (data_ship["year"]=="2018")],data_yaku[(data_yaku["id"]==264) & (data_yaku["year_month"]=="2017-02")]

(Empty DataFrame
 Columns: [year, month, id, efficacy, rx_dom, otc_dom, self_dom, stock_rx, stock_otc, stock_self, year_month]
 Index: [],
    year  month   id      efficacy      rx_agg  rx_pro_agg  rx_pro_dom  \
 0  2017      2  264  鎮痛，鎮痒，収斂，消炎剤  13,986,801  13,672,584  11,907,637   
 
      otc_agg otc_pro_agg otc_pro_dom  self_agg  self_pro_agg  self_pro_dom  \
 0  7,864,793   7,731,867     132,926       0.0           0.0           0.0   
 
   year_month  
 0    2017-02  )

In [183]:
result=data_yaku.merge(data_ship, on=["year","month","id","year_month"], how='left', indicator=True).query('_merge == "left_only"')
result.columns
result=result.loc[result["id"].astype(str).str.len()==3]
print(result.shape)
result.to_csv("left_only.csv",encoding="shift-jis")

(4, 22)


In [184]:
data_yaku=data_yaku.merge(data_ship,how="left",on=["year","month","id"])
data_yaku=data_yaku[data_yaku["id"].astype(str).str.len()>=2]
data_yaku.shape,data_yaku.columns

((34336, 22),
 Index(['year', 'month', 'id', 'efficacy_x', 'rx_agg', 'rx_pro_agg',
        'rx_pro_dom', 'otc_agg', 'otc_pro_agg', 'otc_pro_dom', 'self_agg',
        'self_pro_agg', 'self_pro_dom', 'year_month_x', 'efficacy_y', 'rx_dom',
        'otc_dom', 'self_dom', 'stock_rx', 'stock_otc', 'stock_self',
        'year_month_y'],
       dtype='object'))

In [185]:
data_yaku[data_yaku.duplicated(["year","month","id"])]

,year,month,id,efficacy_x,rx_agg,rx_pro_agg,rx_pro_dom,otc_agg,otc_pro_agg,otc_pro_dom,...,self_pro_dom,year_month_x,efficacy_y,rx_dom,otc_dom,self_dom,stock_rx,stock_otc,stock_self,year_month_y


In [186]:
for col in ['stock_rx','stock_otc', 'stock_self']:
    print(col)
    data_yaku[f'{col}_lag']=data_yaku.groupby(["id"])[col].shift()

stock_rx
stock_otc
stock_self


In [187]:
# len 3
df_3=pd.merge(data_yaku,data_ke_3,left_on="id",right_on="id",how="left")
# sort 
df_3=df_3[df_3["id"].astype(str).str.len()==3]
df_3.sort_values(by=["year","id","release_year"]).reset_index(drop=1).to_csv("../merged/len3_prod_mon.csv",encoding="shift-jis")
df_3

,year,month,id,efficacy_x,rx_agg,rx_pro_agg,rx_pro_dom,otc_agg,otc_pro_agg,otc_pro_dom,...,stock_rx_lag,stock_otc_lag,stock_self_lag,Approved,KEGG,Ingredient,date,release_year,成分,variable
1,2009,1,111,全身麻酔剤,1341311,596054,596054,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009,1,112,催眠鎮静剤，抗不安剤,6526356,5458099,1438569,186967,178591,8376,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009,1,113,抗てんかん剤,2528330,2528330,1762856,12624,12624,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009,1,114,解熱鎮痛消炎剤,9749959,9528474,6804286,1328236,1328236,0,...,NaN,NaN,NaN,1994.0,D00127,Piroxicam,1995/1/24,1995.0,ピロキシカム,id1
5,2009,1,114,解熱鎮痛消炎剤,9749959,9528474,6804286,1328236,1328236,0,...,NaN,NaN,NaN,1994.0,D00132,Ketoprofen,1995/2/10,1995.0,ケトプロフェン,id1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45974,2023,4,749,その他の体外診断用医薬品,-,-,-,-,-,-,...,46181,-,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45976,2023,4,799,他に分類されない治療を主目的としない医薬品,"1,297,493","1,297,493","726,596","576,769","179,548",-,...,1497563,170752,0.0,2001.0,D03365,Nicotine,2001/9/10,2001.0,ニコチン,id1
45978,2023,4,811,あへんアルカロイド系麻薬,"1,535,740","1,535,740","1,535,740",-,-,-,...,6667411,-,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45979,2023,4,812,コカアルカロイド系製剤,-,-,-,-,-,-,...,-,-,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
# len 2
df_2=pd.merge(data_yaku,data_ke_2,left_on="id",right_on="id",how="left")
df_2=df_2[df_2["id"].astype(str).str.len()==2]
df_2.sort_values(by=["year","id","release_year"]).reset_index(drop=1).to_csv("../merged/len2_prod_mon.csv",encoding="shift-jis")
df_2

,year,month,id,efficacy_x,rx_agg,rx_pro_agg,rx_pro_dom,otc_agg,otc_pro_agg,otc_pro_dom,...,stock_rx_lag,stock_otc_lag,stock_self_lag,Approved,KEGG,Ingredient,date,release_year,成分,variable
0,2009,1,11,中枢神経系用薬,53505512,42488269,28835520,3326776,3300439,26337,...,NaN,NaN,NaN,1994.0,D00127,Piroxicam,1995/1/24,1995.0,ピロキシカム,id1
1,2009,1,11,中枢神経系用薬,53505512,42488269,28835520,3326776,3300439,26337,...,NaN,NaN,NaN,1994.0,D00132,Ketoprofen,1995/2/10,1995.0,ケトプロフェン,id1
2,2009,1,11,中枢神経系用薬,53505512,42488269,28835520,3326776,3300439,26337,...,NaN,NaN,NaN,2002.0,D01578,Pranoprofen,2004/2/16,2004.0,テルビナフィン塩酸塩,id1
3,2009,1,11,中枢神経系用薬,53505512,42488269,28835520,3326776,3300439,26337,...,NaN,NaN,NaN,2009.0,D00904,Diclofenac sodium,2009/4/28,2009.0,ジクロフェナクナトリウム,id1
4,2009,1,11,中枢神経系用薬,53505512,42488269,28835520,3326776,3300439,26337,...,NaN,NaN,NaN,2010.0,D01709,Loxoprofen sodium hydrate,2011/1/21,2011.0,ロキソプロフェンナトリウム水和物,id1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47083,2023,4,72,診断用薬（体外診断用医薬品を除く。）,"5,858,072","3,036,355","544,038",-,-,-,...,15381632,-,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47087,2023,4,73,公衆衛生用薬,"680,337","680,337","310,919","1,431,176","1,424,896","1,098,078",...,712843,6771517,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47100,2023,4,79,その他の治療を主目的としない医薬品,"1,297,493","1,297,493","726,596","576,769","179,548",-,...,1497563,170752,0.0,2001.0,D03365,Nicotine,2001/9/10,2001.0,ニコチン,id1
47102,2023,4,81,アルカロイド系麻薬（天然麻薬）,"1,535,740","1,535,740","1,535,740",-,-,-,...,6667411,-,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [189]:
!pwd

/c/Users/hakos/GDrive/Downloads/Research/data/yakuji
